In [2]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import torch
import os
import torchvision
from scipy.ndimage import binary_erosion
import matplotlib.colors as colors

python e_inference_myocard_segmentation.py --input_path "L:\basic\Personal Archive\E\emquist\parsing_MRI\new_test_seg\lge_for_merged" --output_path "L:\basic\Personal Archive\E\emquist\parsing_MRI\new_test_seg\myo_predictions" --load_checkpoint "L:\basic\diva1\Onderzoekers\DEEP-RISK\DEEP-RISK\models_Roel\tb_logs_segmentation_myo\unet_myo_PSIR\version_1\checkpoints\epoch=119-step=6960.ckpt" --center_crop 200 --resize 224 --input_extension nii.gz --files_must_contain psir

# Visualize LGE myocard

In [ ]:
def visualize_stack(img, *segmentations):
    """ Visualize the images and their segmentations for debugging purposes.
    number of arguments for segmentations is variable, each type will be shown
    in a different column, e.g. both myocardium and fibrosis segmentations.

    img and segmentations should be either (D, C, H, W) or (C, D, H, W),
    with number of channels C=1, which will be converted to (D, C, H, W).
    """
    #C, D, H, W = img.shape
    #img = img.view(D, C, H, W)
    #myo_seg = myo_seg.view(D, C, H, W)
    print(f"{img.shape=}")
    print(f"{img.mean()=}")
    img = img.squeeze().unsqueeze(1) # set channel dimension at dim 1
    grid_img = torchvision.utils.make_grid(img, nrow=4, normalize=True, padding=2).detach().cpu()
    num_rows = len(segmentations)+1
    fig, axs = plt.subplots(1, num_rows, figsize=(8*num_rows, 8))
    fig.colorbar(cm.ScalarMappable(cmap='bwr'), ax=axs.ravel().tolist())
    axs[0].imshow(grid_img[0,:,:], cmap='gray')
    axs[0].set_title("LGE Image")
    for i, segmentation in enumerate(segmentations):
        segmentation = segmentation.squeeze().unsqueeze(1) # set channel dimension at dim 1
        assert img.shape == segmentation.shape, f"{img.shape=} {segmentation.shape=}"
        grid_seg = torchvision.utils.make_grid(segmentation, nrow=4, padding=2).cpu().detach()
        grid_seg = np.ma.masked_where(grid_seg <= 0.2, grid_seg)
        axs[i+1].imshow(grid_img[0,:,:], cmap='gray', vmin=0, vmax=1)
        axs[i+1].imshow(grid_seg[0,:,:], cmap='bwr', alpha=0.8, vmin=0, vmax=1)

    for i in range(num_rows):
        axs[i].set_axis_off()
    plt.show()
    return

In [3]:
myo_f = sitk.ReadImage(f"L:\\basic\Personal Archive\E\emquist\parsing_MRI\\final_test_seg\myo_pred_all_lge/11_sagittal_tfi_psir_single-shot_10sl_ipat_sa_psir_(192,144)_myo_pred.nrrd")
# segmentation = sitk.PermuteAxes(segmentation, [2,0,  1])
# Convert the SimpleITK image to a NumPy array
myo_segmentation = sitk.GetArrayFromImage(myo_f)
myo_segmentation = torch.Tensor(myo_segmentation.astype('float'))

# visualize_stack(img, myo_segmentation)
plt.imshow(myo_segmentation[6, ...], cmap='gray')
# plt.title('Your Image Title')
plt.axis('off')  # Turn off axis labels
plt.show()

: 

In [ ]:
img_file = '35_sagittal_sa_psir_(192,144)'
# img_file = ('_').join(fib_file.split('_')[:-2])


# Load the NRRD file
# fib_f = sitk.ReadImage(f"L:/basic\Personal Archive\E\emquist\parsing_MRI/new_test_seg/fib_predictions/{img_file}_fib_pred.nrrd")
# # segmentation = sitk.PermuteAxes(segmentation, [2,0,  1])
# # Convert the SimpleITK image to a NumPy array
# fib_segmentation = sitk.GetArrayFromImage(fib_f)
# fib_segmentation = torch.Tensor(fib_segmentation.astype('float'))

# Load the NRRD file
# img_f = sitk.ReadImage(f"L:/basic\Personal Archive\E\emquist\parsing_MRI/new_test_seg/lge_for_merged/{img_file}.nii.gz")
# # Convert the SimpleITK image to a NumPy array
# # img = sitk.PermuteAxes(img, [2,0,  1])
# img = sitk.GetArrayFromImage(img_f)
# img = torch.Tensor(img.astype('float'))

# Load the NRRD file
myo_f = sitk.ReadImage(f"L:\\basic\Personal Archive\E\emquist\parsing_MRI\\final_test_seg\myo_pred_all_lge/11_sagittal_tfi_psir_single-shot_10sl_ipat_sa_psir_(192,144)_myo_pred.nrrd")
# segmentation = sitk.PermuteAxes(segmentation, [2,0,  1])
# Convert the SimpleITK image to a NumPy array
myo_segmentation = sitk.GetArrayFromImage(myo_f[6, ...], cmap='gray')
myo_segmentation = torch.Tensor(myo_segmentation.astype('float'))

# visualize_stack(img, myo_segmentation)
plt.imshow(myo_segmentation[10])
# plt.title('Your Image Title')
plt.axis('off')  # Turn off axis labels
plt.show()

In [ ]:
# def extract_segmentation_boundaries(segmentation_array):
#     # Perform binary thresholding to obtain a binary mask
#     threshold = 0.95  # Adjust this threshold based on your segmentation values
#     binary_mask = (segmentation_array < threshold) > 0.05

#     # Perform morphological erosion to obtain the boundaries
#     eroded_mask = binary_erosion(binary_mask)
#     outer_boundary = binary_mask.astype(np.uint8) - eroded_mask.astype(np.uint8)
#     rest = segmentation_array - outer_boundary
#     return outer_boundary

# Read the NIfTI image
# nifti_image = sitk.ReadImage('original.nii.gz')
nifti_array = sitk.GetArrayFromImage(img_f)

# Read the NRRD segmentation
# nrrd_image = sitk.ReadImage('segmentation.nrrd')
myo_array = sitk.GetArrayFromImage(myo_f)
myo_masked = np.ma.masked_outside(myo_array, 0.03, 0.97)
# outer_boundary = extract_segmentation_boundaries(myo_array)
# fib_array = sitk.GetArrayFromImage(fib_f)
# fib_masked = np.ma.masked_where(fib_array < 0.5, fib_array)

masked_color = 'blue'
cmap = colors.ListedColormap([masked_color])
norm = colors.Normalize(vmin=0, vmax=1)

# Plot the original NIfTI image and the segmentation boundaries
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(nifti_array[6, ...], cmap='gray')
ax[0].set_title('Original NIfTI Image')
ax[1].imshow(nifti_array[6, ...], cmap='gray')
ax[1].imshow(myo_masked[6, ...], cmap='plasma', norm=norm, alpha=0.8)
# ax[1].imshow(fib_masked[6,:,:], cmap='bwr', vmin=0, vmax=1)
ax[1].set_title('Segmentation Boundaries')
ax[1].set_axis_off()
ax[0].set_axis_off()

plt.show()
plt.savefig(f"L:/basic\Personal Archive\E\emquist\parsing_MRI/test_mri_seg/vis_try.png")


In [ ]:
def get_pixel_intensities(niftis, myo_pred, norm_scan_intenities):
    blood_pool_intensities, heart_muscle_intensities = [], []

    for i, scan in enumerate(niftis):
        #         plot_annotated_scan(scan)
        threshold = 0.6
        heart_muscle_mask = (myo_pred[i] > threshold) 
        dicom_img = niftis[i]#.astype(float)
        if norm_scan_intenities:
            try:
                dicom_img -= dicom_img[heart_muscle_mask].min()
                dicom_img /= dicom_img[heart_muscle_mask].max()
            except:
                continue
        heart_muscle_intensities += dicom_img[heart_muscle_mask].tolist()

    return dicom_img, heart_muscle_intensities

In [ ]:
from skimage import measure

In [ ]:
def visualize_stack(img_file, save):
    """ Visualize the images and their segmentations for debugging purposes.
    number of arguments for segmentations is variable, each type will be shown
    in a different column, e.g. both myocardium and fibrosis segmentations.

    img and segmentations should be either (D, C, H, W) or (C, D, H, W),
    with number of channels C=1, which will be converted to (D, C, H, W).
    """
    #C, D, H, W = img.shape
    #img = img.view(D, C, H, W)
    #myo_seg = myo_seg.view(D, C, H, W)
    # print(f"{img.shape=}")
    # print(f"{img.mean()=}")
    out_path = "L:/basic\Personal Archive\E\emquist\parsing_MRI/new_test_seg/visualizations/"
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    # # Load the NRRD file
    # fib_f = sitk.ReadImage(f"L:/basic\Personal Archive\E\emquist\parsing_MRI/test_mri_seg/fib_predictions/{img_file}_fib_pred.nrrd")
    # fib_segmentation = sitk.GetArrayFromImage(fib_f)
    # fib_segmentation = torch.Tensor(fib_segmentation.astype('float'))

    # Load the NRRD file
    img_f = sitk.ReadImage(f"L:/basic\Personal Archive\E\emquist\parsing_MRI/new_test_seg/lge_for_merged/{img_file}.nii.gz")
    img = sitk.GetArrayFromImage(img_f)
    img = torch.Tensor(img.astype('float'))

    # Load the NRRD file
    myo_f = sitk.ReadImage(f"L:/basic\Personal Archive\E\emquist\parsing_MRI/new_test_seg/myo_predictions/{img_file}_myo_pred.nrrd")
    myo_seg = sitk.GetArrayFromImage(myo_f)
    myo_segmentation = torch.Tensor(myo_seg.astype('float'))
    

    img = img.squeeze().unsqueeze(1) # set channel dimension at dim 1
    grid_img = torchvision.utils.make_grid(img, nrow=4, normalize=True, padding=2).detach().cpu()
    num_rows = 2
    fig, axs = plt.subplots(1, num_rows, figsize=(8*num_rows, 8))
    # fig.colorbar(cm.ScalarMappable(cmap='bwr'), ax=axs.ravel().tolist())
    axs[0].imshow(grid_img[0,:,:], cmap='gray')
    axs[0].set_title("LGE Image")
    for i, myo in enumerate([myo_segmentation]):
        myo = myo.squeeze().unsqueeze(1) # set channel dimension at dim 1
        assert img.shape == myo.shape, f"{img.shape=} {myo.shape=}"
        
        myo = torchvision.utils.make_grid(myo, nrow=4, padding=2).cpu().detach()
        original_image = (grid_img[0, :, :]).numpy()
        masked_image = original_image
        # Compute the contour of the myocardium segmentation
        contours = measure.find_contours((myo[0, :, :]).numpy(), 0.5) 
        # Display the original image
        # Plot the myocardium contour
        for contour in contours:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=.5, color='red')

        # Create a mask from the contour
        mask = np.zeros_like(original_image)
        for contour in contours:
            contour = np.round(contour).astype(int)
            mask[contour[:, 0], contour[:, 1]] = 1

        # Set the region outside the contour to a lower transparency value
        outside = np.where(mask == 0)
        masked_image[outside] *= 0.7  # Adjust the transparency value as needed

        # Overlay the modified original image on top of the plo
        # fib = fib_segmentation.squeeze().unsqueeze(1)
        # fib = torchvision.utils.make_grid(fib, nrow=4, padding=2).cpu().detach()
        # fib_masked = np.ma.masked_where(fib < 0.1, fib)# grid_seg = np.ma.masked_where(grid_seg <= 0, grid_seg)

        axs[i+1].imshow(masked_image, cmap="gray", alpha=0.7)
        # axs[i+1].imshow(fib_masked[0,:,:], cmap='plasma')

    for i in range(num_rows):
        axs[i].set_axis_off()
        
    if save:
        plt.savefig(out_path + f"{img_file}.png")
    else:
        plt.show()
    plt.close()

In [ ]:
# from pathlib import Path
# FIB_DIR = Path("L:/basic\Personal Archive\E\emquist\parsing_MRI/test_mri_seg/fib_predictions")
# must_contain = 'psir'
# extension = 'nrrd'
# fib_files =  [f for f in sorted(FIB_DIR.glob(f'*{must_contain}*.{extension}'))]
# for fib_file in fib_files:
#     fib_file = ("_").join((Path(fib_file).stem).split("_")[:-2])
#     # print(fib_file)
#     visualize_stack(fib_file, True)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from collections import defaultdict, OrderedDict, Counter
import copy
import time
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib.image as mpimg
from datetime import date, datetime
import uuid
import os
from PIL import Image
import io
import pickle

In [ ]:
IN_DIR_NIFTI = Path("L:/basic\Personal Archive\E\emquist\parsing_MRI/new_test_seg/lge_for_merged")
IN_DIR_MYO = Path("L:/basic\Personal Archive\E\emquist\parsing_MRI/new_test_seg/myo_predictions")
extension_myo = 'nrrd'
mcl = 'psir'
extension_nifti = 'nii.gz'
# nifti_files = [sitk.GetArrayFromImage(sitk.ReadImage(x)) for x in sorted(IN_DIR_NIFTI.glob(f'*{mcl}*.{extension_nifti}'))]
# myo_pred = [sitk.GetArrayFromImage(sitk.ReadImage(x)) for x in sorted(IN_DIR_MYO.glob(f'*{mcl}*.{extension_myo}'))]

In [ ]:
def plot_pixel_intensities_hist(niftis, myo_pred, norm_scan_intenities):
    heart_muscle_intensities = get_pixel_intensities(niftis, myo_pred, norm_scan_intenities)
    plt.figure(figsize=(14, 8))
    plt.hist([heart_muscle_intensities], bins=100)
    plt.legend(["heart muscle"])
    plt.xlabel("pixel intensity")
    norm_scan_intenities_txt = "normed" if norm_scan_intenities else ""
    plt.title("Hisotgram of pixel intensities " + norm_scan_intenities_txt)

In [ ]:
# plot_pixel_intensities_hist(nifti_files, myo_pred, norm_scan_intenities=False)

In [ ]:
plot_pixel_intensities_hist(nifti_files, myo_pred, norm_scan_intenities=True)